## Containerize model server & ML API with Docker

### Build and push docker images to docker-repository
One common way of distributing this model API server for production deployment, is via Docker containers. 

In [2]:
!sudo chmod 777 /var/run//docker.sock

In [3]:
# Replace {docker_username} with your Docker Hub username
docker_username = 'kumarvc'
# Build and tage docker images
!docker build -t {docker_username}/fastapi-placement-classifier:1.0.3 .
# Push Docker images to Docker hub (Public repository)
!docker push {docker_username}/fastapi-placement-classifier:1.0.3

Sending build context to Docker daemon  4.354MB
Step 1/5 : FROM tiangolo/uvicorn-gunicorn-fastapi:python3.7
 ---> e2f19ac0b4e3
Step 2/5 : COPY /app/requirements.txt /app/
 ---> Using cache
 ---> 9e361e5ae998
Step 3/5 : RUN pip install -r /app/requirements.txt
 ---> Using cache
 ---> aeb9ca75bf7f
Step 4/5 : COPY ./model /model/
 ---> Using cache
 ---> a99fc343dab0
Step 5/5 : COPY ./app /app
 ---> Using cache
 ---> 7b6a67337a31
Successfully built 7b6a67337a31
Successfully tagged kumarvc/fastapi-placement-classifier:1.0.3
The push refers to repository [docker.io/kumarvc/fastapi-placement-classifier]

da0cc980: Preparing 
84f549ea: Preparing 
8885b7d9: Preparing 
949c3b04: Preparing 
2331eddf: Preparing 
dd42a306: Preparing 
5e087746: Preparing 
a89f95f7: Preparing 
960321f5: Preparing 
c8cc20a5: Preparing 
85a516c9: Preparing 
ab020550: Preparing 
5ea49213: Preparing 
28316107: Preparing 
0ec29f78: Preparing 
cecc2826: Preparing 
81fca4b7: Preparing 
92e98337: Preparing 
306e673e: Prepari

## Run containerize model server - Ready to receive request

In [ ]:
# For full documentation please refere https://github.com/tiangolo/uvicorn-gunicorn-fastapi-docker
!docker run --name fastapi-server -d -p 80:80 -e WORKERS_PER_CORE="3" {docker_username}/fastapi-placement-classifier:1.0.3

71ba710def3b1891cda3116e7220554b19241522bae3520dbf29f36129d3490a


## Send request to Local FastAPI model server

The Fast API server also provides a simple web UI dashboard (Swagger). Go to http://0.0.0.0:80/docs in the browser and use the Web UI to send prediction request:

### Send Batch of test request from Python program or from python command prompt

In [5]:
import requests
data = {
  "sl_no": 112,
  "ssc_p": 84.0,
  "hsc_p": 90.9,
  "degree_p": 64.5,
  "etest_p": 86.04,
  "mba_p": 59.42,
  "gender": "M",
  "ssc_b": "Others",
  "hsc_b": "Others",
  "hsc_s": "Science",
  "degree_t": "Sci&Tech",
  "workex": "No",
  "specialisation": "Mkt&Fin"
}
response = requests.post("http://0.0.0.0:80/predict", json=data)
print(response.text)

{"prediction":["Placed"]}


In [6]:
data = {
  "sl_no": 113,
  "ssc_p": 52.0,
  "hsc_p": 57.0,
  "degree_p": 50.8,
  "etest_p": 67.0,
  "mba_p": 62.79,
  "gender": "M",
  "ssc_b": "Central",
  "hsc_b": "Central",
  "hsc_s": "Commerce",
  "degree_t": "Comm&Mgmt",
  "workex": "No",
  "specialisation": "Mkt&HR"
}
response = requests.post("http://0.0.0.0:80/predict", json=data)
print(response.text)

{"prediction":["Not Placed"]}


## Check Docker logs

In [7]:
!docker logs 1ff1aae77f

Checking for script in /app/prestart.sh
Running script /app/prestart.sh
Running inside /app/prestart.sh, you could add migrations to this file, e.g.:

#! /usr/bin/env bash

# Let the DB start
sleep 10;
# Run migrations
alembic upgrade head

[2021-01-03 12:25:31 +0000] [1] [INFO] Starting gunicorn 20.0.4
[2021-01-03 12:25:31 +0000] [1] [INFO] Listening at: http://0.0.0.0:80 (1)
[2021-01-03 12:25:31 +0000] [1] [INFO] Using worker: uvicorn.workers.UvicornWorker
[2021-01-03 12:25:31 +0000] [8] [INFO] Booting worker with pid: 8
[2021-01-03 12:25:31 +0000] [9] [INFO] Booting worker with pid: 9
[2021-01-03 12:25:31 +0000] [10] [INFO] Booting worker with pid: 10
[2021-01-03 12:25:31 +0000] [11] [INFO] Booting worker with pid: 11
[2021-01-03 12:25:32 +0000] [12] [INFO] Booting worker with pid: 12
[2021-01-03 12:25:32 +0000] [13] [INFO] Booting worker with pid: 13
[2021-01-03 12:25:32 +0000] [14] [INFO] Booting worker with pid: 14
[2021-01-03 12:25:32 +0000] [15] [INFO] Booting worker with pid: 

## Stop and delete the running MLAPI container

In [5]:
!docker stop 1ff1aae77f61ca43317
!docker rm 1ff1aae77f61ca43317

1ff1aae77f61ca43317
1ff1aae77f61ca43317
